In [ ]:
#####################################################################################################
#####################################################################################################
# 1_Functions
#####################################################################################################
#####################################################################################################

In [ ]:
# Creation of a function that retrieves all the indexes of a character string in a text file
def all_occurences(file, str):
    initial = 0
    while True:
        initial = file.find(str, initial)
        if initial == -1: return
        yield initial
        initial += len(str)
        

In [ ]:
# Import function for text file from Zotero software
def import_zot(file):
    file = open(file,'r', encoding="utf-8")
    text = file.read()
    file.close()
    
    # We put the text entirely in lowercase
    text=text.lower()
    
    # Creation of a list indicating the words to delete
    listeMotsASupprimer = ['{\\textgreater}','{\\textless}']

    # For each word from the list: "listeMotsASupprimer", python will store it in the variable 
    # "motASupprimer", and do the replacement processing with this word
    for motASupprimer in listeMotsASupprimer:
        text=text.replace(motASupprimer,'') # no space to not distort the text
    
    # Calculation of the number of articles in the input text file
    nb_article=0
    nb_article=text.count('@article')
    
    # Creation of a list containing all the indexes at the start of the article block
    liste_index_art=list(all_occurences(text,'@article'))
    
    # Creation of an article list which contains an article block for each element of the list
    article=[]
    for i in range(nb_article-1):
        s=text[liste_index_art[i]:liste_index_art[i+1]]
        article.append(s)
    # We add the last abstract
    article.append(text[liste_index_art[nb_article-1]:])

    # Isolation of the backgrounds then creation of an abstract list, with an abstract by element 
    # or "abstract absent" in the case where there is no abstract
    index_acf=0
    index_ac=0
    abstract=[]

    for a in article:
        index_a=a.find('abstract =', )
        index_ac=a.find('{',index_a,)
        index_acf=a.find('}', index_ac,)
        if index_a!=-1:
            t=a[index_ac+1:index_acf]
            abstract.append(t)
        else:
            abstract.append("absent abstract")              
    return abstract


In [ ]:
# Import function for text files from Cochrane
def import_coch(file):
    file = open(file,'r', encoding="utf-8")
    text = file.read()
    file.close()

    # We put the text entirely in lowercase
    text=text.lower()

    # Calculation of the number of articles in the input text file
    nb_article=0
    nb_article=text.count('record #')

    # Creation of a list containing all the indexes at the start of the article block
    liste_index_art=list(all_occurences(text,'record #'))

    # Creation of an article list which contains an article block for each element of the list
    article=[]
    for i in range(nb_article-1):
        s=text[liste_index_art[i]:liste_index_art[i+1]]
        article.append(s)
    # We add the larst abstract
    article.append(text[liste_index_art[nb_article-1]:])

    # Isolation of the backgrounds then creation of an abstract list, with an abstract by element 
    # or "abstract absent" in the case where there is no abstract
    index_acf=0
    index_ac=0
    abstract=[]

    for a in article:
        index_a=a.find('ab: ', )+2
        index_acf=a.find('us: ', index_a,)
        if index_a!=-1:
            t=a[index_a+1:index_acf]
            abstract.append(t)

        else:
            abstract.append("absent abstract")
    return abstract


In [ ]:
# Creation of a clean function to clean all abstracts
import nltk
nltk.download('all')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized


In [ ]:
# Function which generates n-gramm from a list of words
def generate_ngrams(words_list, n):
    ngrams_list = []
 
    for num in range(0, len(words_list)):
        ngram = ' '.join(words_list[num:num + n])
        ngrams_list.append(ngram) 
    return ngrams_list


In [ ]:
# Creation of a function that takes as input a list of words and a list of keywords to find
# The function will output a set (without duplicates) of keywords
def recherche_mot_cle(liste_mot, liste_mot_cle):
    select=[]
       
    for mot in liste_mot:
        if mot in liste_mot_cle:
            select.append(mot)
    # We transform the list as a whole which allows us to remove duplicates
    ens=set(select)           
    return ens


In [ ]:
# We are looking to enrich the word list of each abstract with n-grams
def gen_liste_ngrams(doc,n):
    liste_ngram=[]
    for i in range(1,n+1):
        liste_ngram=liste_ngram+(generate_ngrams(doc,i))        
    return liste_ngram


In [ ]:
# Functions for creating a word cloud

# 1/ For single word:
# we enter in parameter the list and the number of words that we want to appear
def wordcloud(liste, n, color):
    words_dict=dict(nltk.FreqDist(liste))
    # Generate word clouds and save them as jpg images
    WC_height = 2000
    WC_width = 3000
    WC_max_words = n
    wordCloud = WordCloud(max_font_size=750,max_words=WC_max_words, height=WC_height, width=WC_width, background_color="white",  colormap=color)
    wordCloud.generate_from_frequencies(words_dict)
    plt.imshow(wordCloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    return wordCloud


# 2/ For bigrams, you enter in parameter the list and the number of words you want to appear
def wordcloud_bigram(liste, n, color):
    # Calculation of bigrams with the nltk and countVectorizer function
    bigrams_list = list(nltk.bigrams(liste))
    dictionary2 = [' '.join(tup) for tup in bigrams_list]
    words_dict=dict(nltk.FreqDist(dictionary2))

    # Generate word clouds and save them as jpg images
    WC_height = 2000
    WC_width = 3000
    WC_max_words = n
    wordCloud = WordCloud(max_font_size=750,max_words=WC_max_words, height=WC_height, width=WC_width, background_color="white",  colormap=color)
    wordCloud.generate_from_frequencies(words_dict)
    plt.imshow(wordCloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    return wordCloud
    
# 3/ For trigrams, enter the list and the number of words you want to appear in parameters
def wordcloud_trigram(liste, n, color):
    trigrams_list = list(nltk.trigrams(liste))
    dictionary2 = [' '.join(tup) for tup in trigrams_list]
    words_dict=dict(nltk.FreqDist(dictionary2))

    # Generate word clouds and save them as jpg images
    WC_height = 2000
    WC_width = 3000
    WC_max_words = n
    wordCloud = WordCloud(max_font_size=750,max_words=WC_max_words, height=WC_height, width=WC_width, background_color="white",  colormap=color)
    wordCloud.generate_from_frequencies(words_dict)
    plt.imshow(wordCloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    return wordCloud
